In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
df_train=pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")
df_test=pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")
df_sybmission=pd.read_csv("../input/tabular-playground-series-oct-2021/sample_submission.csv")


In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.dtypes

In [ ]:
df_train.dtypes.value_counts()

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
df_train.columns

In [ ]:
df_train.describe

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import warnings
# Matplotlib e Seaborn
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True)
# Plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode()

In [ ]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:


%%time
missing_data(df_train)



In [ ]:
%%time
missing_data(df_test)

In [ ]:
def plot_feature_scatter(df1, df2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(4,4,figsize=(14,14))

    for feature in features:
        i += 1
        plt.subplot(4,4,i)
        plt.scatter(df1[feature], df2[feature], marker='+')
        plt.xlabel(feature, fontsize=9)
    plt.show()

In [ ]:
df_train.shape

In [ ]:
# select the float columns
df_float = df_train.select_dtypes(include=[np.float64])
# select the int columns
df_int = df_train.select_dtypes(include=[np.int64])

In [ ]:
df_float

In [ ]:
df_int[:-1]

In [ ]:
df_float.columns

In [ ]:
features = ['f0', 'f1','f2','f3', 'f4', 'f5', 'f6', 'f7', 
           'f8', 'f9', 'f10','f11','f12', 'f13', 'f14', 'f15', 
           ]
plot_feature_scatter(df_float[::20],df_float[::20], features)



In [ ]:
sns.countplot(df_train['target'], palette='Set3')

In [ ]:
df_train['target'].value_counts()

In [ ]:
print("There are {}% target values with 1".format(100 * df_train["target"].value_counts()[1]/df_train.shape[0]))


In [ ]:
print("There are {}% target values with 0".format(100 * df_train["target"].value_counts()[0]/df_train.shape[0]))

In [ ]:
features=df_float.columns

In [ ]:
plt.figure(figsize=(16,6))
features = df_train.columns.values[2:202]
plt.title("Distribution of mean values per row in the train and test set")
sns.distplot(df_train[features].mean(axis=1),color="green", kde=True,bins=120, label='train')
sns.distplot(df_test[features].mean(axis=1),color="blue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per column in the train and test set")
sns.distplot(df_train[features].mean(axis=0),color="magenta",kde=True,bins=120, label='train')
sns.distplot(df_test[features].mean(axis=0),color="darkblue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of std values per row in the train and test set")
sns.distplot(df_train[features].std(axis=1),color="black", kde=True,bins=120, label='train')
sns.distplot(df_test[features].std(axis=1),color="red", kde=True,bins=120, label='test')
plt.legend();plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of std values per column in the train and test set")
sns.distplot(df_train[features].std(axis=0),color="blue",kde=True,bins=120, label='train')
sns.distplot(df_test[features].std(axis=0),color="green", kde=True,bins=120, label='test')
plt.legend(); plt.show()

In [ ]:
t0 = df_train.loc[df_train['target'] == 0]
t1 = df_train.loc[df_train['target'] == 1]
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per row in the train set")
sns.distplot(t0[features].mean(axis=1),color="red", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].mean(axis=1),color="blue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per column in the train set")
sns.distplot(t0[features].mean(axis=0),color="green", kde=True,bins=120, label='target = 0')
sns.distplot(t1[features].mean(axis=0),color="darkblue", kde=True,bins=120, label='target = 1')
plt.legend(); plt.show()


Distribution of min and max

Let's check the distribution of min per row in the train and test set.


In [ ]:
plt.figure(figsize=(16,6))

plt.title("Distribution of min values per row in the train and test set")
sns.distplot(df_train[features].min(axis=1),color="red", kde=True,bins=120, label='train')
sns.distplot(df_test[features].min(axis=1),color="orange", kde=True,bins=120, label='test')
plt.legend()
plt.show()

We calculate now the correlations between the features in train set.
The following table shows the first 10 the least correlated features.

**Correlations with all the features( Float and Int both)**

In [ ]:
%%time
correlations = df_train.corr().abs().unstack().sort_values(kind="quicksort").reset_index()
correlations = correlations[correlations['level_0'] != correlations['level_1']]
correlations.head(10)

**Correlations with only the Float features**

In [ ]:
%%time
correlations = df_train[features].corr().abs().unstack().sort_values(kind="quicksort").reset_index()
correlations = correlations[correlations['level_0'] != correlations['level_1']]
correlations.head(10)





Let's look to the top most correlated features, besides the same feature pairs.


In [ ]:
correlations.tail(10)

Let's see also the least correlated features.

In [ ]:
correlations.head(10)

Duplicate values¶

Let's now check how many duplicate values exists per columns.

In [ ]:
%%time
unique_max_train = []
unique_max_test = []
for feature in features:
    values = df_train[feature].value_counts()
    unique_max_train.append([feature, values.max(), values.idxmax()])
    values = df_test[feature].value_counts()
    unique_max_test.append([feature, values.max(), values.idxmax()])



In [ ]:
np.transpose((pd.DataFrame(unique_max_train, columns=['Feature', 'Max duplicates', 'Value'])).\
            sort_values(by = 'Max duplicates', ascending=False).head(15))

Let's see also the top 15 number of duplicates values per test set.

In [ ]:
np.transpose((pd.DataFrame(unique_max_test, columns=['Feature', 'Max duplicates', 'Value'])).\
            sort_values(by = 'Max duplicates', ascending=False).head(15))